In [ ]:
import random
from collections import deque
import os
from tqdm.notebook import tqdm, trange
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
!unzip datasets.zip

Archive:  datasets.zip
   creating: CIC-DDoS2019/
  inflating: CIC-DDoS2019/DNS-testing.parquet  
  inflating: CIC-DDoS2019/LDAP-testing.parquet  
  inflating: CIC-DDoS2019/LDAP-training.parquet  
  inflating: CIC-DDoS2019/MSSQL-testing.parquet  
  inflating: CIC-DDoS2019/MSSQL-training.parquet  
  inflating: CIC-DDoS2019/NetBIOS-testing.parquet  
  inflating: CIC-DDoS2019/NetBIOS-training.parquet  
  inflating: CIC-DDoS2019/NTP-testing.parquet  
  inflating: CIC-DDoS2019/Portmap-training.parquet  
  inflating: CIC-DDoS2019/SNMP-testing.parquet  
  inflating: CIC-DDoS2019/Syn-testing.parquet  
  inflating: CIC-DDoS2019/Syn-training.parquet  
  inflating: CIC-DDoS2019/TFTP-testing.parquet  
  inflating: CIC-DDoS2019/UDP-testing.parquet  
  inflating: CIC-DDoS2019/UDP-training.parquet  
  inflating: CIC-DDoS2019/UDPLag-testing.parquet  
  inflating: CIC-DDoS2019/UDPLag-training.parquet  
  inflating: ALLFLOWMETER_HIKARI2021.csv  


In [ ]:
# Load the first dataset (needs to be in the same folder as ipynb)
hikari = pd.read_csv("ALLFLOWMETER_HIKARI2021.csv")
hikari.drop(columns=["Unnamed: 0.1", "Unnamed: 0"], inplace=True)

In [ ]:
# Load the second dataset (needs to be in the same folder as ipynb)
PATH_TO_CIC = "CIC-DDoS2019"
cic_paths = os.listdir(PATH_TO_CIC)
cic_train_paths = [os.path.join(PATH_TO_CIC, path) for path in cic_paths if "-training" in path]
cic_test_paths = [os.path.join(PATH_TO_CIC, path) for path in cic_paths if "-testing" in path]


train_prefixes = [path.split('/')[-1].split('-')[0] for path in cic_train_paths]
test_prefixes = [path.split('/')[-1].split('-')[0] for path in cic_test_paths]

common_prefixes = list(set(train_prefixes).intersection(test_prefixes))

cic_train_paths = [path for path in cic_train_paths if path.split('/')[-1].split('-')[0] in common_prefixes]
cic_test_paths = [path for path in cic_test_paths if path.split('/')[-1].split('-')[0] in common_prefixes]

cic_train = pd.concat([pd.read_parquet(path) for path in cic_train_paths], ignore_index=True)
cic_test = pd.concat([pd.read_parquet(path) for path in cic_test_paths], ignore_index=True)

In [ ]:
# Convert key names from dataset 1 to dataset 2
hikari_to_cic_keys = {
    'flow_duration': 'Flow Duration',
    'fwd_pkts_tot': 'Total Fwd Packets',
    'bwd_pkts_tot': 'Total Backward Packets',
    'fwd_data_pkts_tot': 'Fwd Packets Length Total',
    'bwd_data_pkts_tot': 'Bwd Packets Length Total',
    'fwd_pkts_per_sec': 'Fwd Packets/s',
    'bwd_pkts_per_sec': 'Bwd Packets/s',
    'down_up_ratio': 'Down/Up Ratio',
    'fwd_header_size_tot': 'Fwd Header Length',
    'bwd_header_size_tot': 'Bwd Header Length',
    'flow_pkts_per_sec': 'Flow Packets/s',
    'flow_iat.avg': 'Flow IAT Mean',
    'flow_iat.std': 'Flow IAT Std',
    'flow_iat.max': 'Flow IAT Max',
    'flow_iat.min': 'Flow IAT Min',
    'fwd_iat.tot': 'Fwd IAT Total',
    'fwd_iat.max': 'Fwd IAT Max',
    'fwd_iat.avg': 'Fwd IAT Mean',
    'fwd_iat.std': 'Fwd IAT Std',
    'fwd_iat.min': 'Fwd IAT Min',
    'bwd_iat.tot': 'Bwd IAT Total',
    'bwd_iat.avg': 'Bwd IAT Mean',
    'bwd_iat.max': 'Bwd IAT Max',
    'bwd_iat.min': 'Bwd IAT Min',
    'bwd_iat.std': 'Bwd IAT Std',
    'flow_SYN_flag_count': 'SYN Flag Count',
    'flow_RST_flag_count': 'RST Flag Count',
    'fwd_PSH_flag_count': 'Fwd PSH Flags',
    'bwd_PSH_flag_count': 'Bwd PSH Flags',
    'flow_ACK_flag_count': 'ACK Flag Count',
    'fwd_URG_flag_count': 'Fwd URG Flags',
    'bwd_URG_flag_count': 'Bwd URG Flags',
    'flow_ECE_flag_count': 'ECE Flag Count',
    'fwd_bulk_rate': 'Fwd Avg Bulk Rate',
    'bwd_bulk_rate': 'Bwd Avg Bulk Rate',
    'fwd_bulk_packets': 'Fwd Avg Packets/Bulk',
    'bwd_bulk_packets': 'Bwd Avg Packets/Bulk',
    'fwd_bulk_bytes': 'Fwd Avg Bytes/Bulk',
    'bwd_bulk_bytes': 'Bwd Avg Bytes/Bulk',
    'active.avg': 'Active Mean',
    'active.min': 'Active Min',
    'active.max': 'Active Max',
    'active.std': 'Active Std',
    'idle.avg': 'Idle Mean',
    'idle.min': 'Idle Min',
    'idle.max': 'Idle Max',
    'idle.std': 'Idle Std',
    'fwd_init_window_size': 'Init Fwd Win Bytes',
    'bwd_init_window_size': 'Init Bwd Win Bytes',
    'fwd_subflow_pkts': 'Subflow Fwd Packets',
    'bwd_subflow_pkts': 'Subflow Bwd Packets',
    'fwd_subflow_bytes': 'Subflow Fwd Bytes',
    'bwd_subflow_bytes': 'Subflow Bwd Bytes',
    'Label': 'Label'
}

In [ ]:
# Remove all the keys from dataset 1 that are not overlapping between 2 datasets
hikari_filtered = hikari.loc[:, hikari.columns.intersection(hikari_to_cic_keys.keys())]
# Rename all the overlapping columns in dataset 1 to match names with dataset 2
hikari_cic = hikari_filtered.rename(columns=hikari_to_cic_keys)

In [ ]:
# Remove all the keys from dataset 2 that are not overlapping between 2 datasets
cic_train = cic_train.loc[:, cic_train.columns.intersection(hikari_to_cic_keys.values())]
cic_test = cic_test.loc[:, cic_test.columns.intersection(hikari_to_cic_keys.values())]

In [ ]:
# Split the first dataset on train and test (75% - train, 25% - test)
hikari_train = hikari_cic.sample(frac=0.75, random_state=42)
hikari_test = hikari_cic.drop(hikari_train.index)

In [ ]:
# Change labels from dataset 2 to be binary (Benign/Attack)
label_conversion_dict = cic_train.Label.unique().tolist() + cic_test.Label.unique().tolist()
label_conversion_dict = list(set(label_conversion_dict))
label_conversion_dict = {
    key: 0 if key == "Benign" else 1 for key in label_conversion_dict
}

cic_train["Label"] = cic_train["Label"].replace(label_conversion_dict)
cic_test["Label"] = cic_test["Label"].replace(label_conversion_dict)

<ipython-input-9-3386602664>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cic_train["Label"] = cic_train["Label"].replace(label_conversion_dict)
<ipython-input-9-3386602664>:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cic_test["Label"] = cic_test["Label"].replace(label_conversion_dict)


In [ ]:
# Combine 2 datasets
combined_train = pd.concat([cic_train, hikari_train])
combined_test = pd.concat([cic_test, hikari_test])

In [ ]:
# Separate labels from the features
labels_train = combined_train["Label"]
labels_test = combined_test["Label"]

combined_train.drop(columns=["Label"], inplace=True)
combined_test.drop(columns=["Label"], inplace=True)

In [ ]:
# Fit the scaler to normalize the features
scaler = StandardScaler().fit(combined_train)

In [ ]:
# Normalize both train and test features
train_features = scaler.transform(combined_train)
test_features = scaler.transform(combined_test)

In [ ]:
# Get size of a feature vector
input_dim = train_features.shape[1]

# Get device for running the models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Latend dimension size of the autoencoder
latent_dim = 16

In [ ]:
# Drop the index from the labels
train_labels = labels_train.values
test_labels = labels_test.values

In [ ]:
# Logic to make sequences from individual requests
def make_sequences(features: np.ndarray,
                   labels: np.ndarray,
                   seq_len: int = 20,
                   stride: int = 10,
                   label_strategy: str = 'any',
                   threshold: float = 0.0):
    windows = []
    sequence_labels = []
    num_flows = features.shape[0]

    for start in range(0, num_flows - seq_len + 1, stride):
        win = features[start:start + seq_len]
        win_labels = labels[start:start + seq_len]

        if label_strategy == 'any':
            # Label as Attack if any flow in the window is malicious
            seq_label = int(win_labels.any())
        elif label_strategy == 'threshold':
            # Label as Attack if proportion > threshold
            seq_label = int(win_labels.mean() > threshold)
        else:
            raise ValueError(f"Unknown label_strategy: {label_strategy}")

        windows.append(win)
        sequence_labels.append(seq_label)

    sequences = np.stack(windows, axis=0)
    sequence_labels = np.array(sequence_labels, dtype=np.int64)
    return sequences, sequence_labels

In [ ]:
# Make sequences out of the datasets
train_features, train_labels = make_sequences(train_features, labels_train.to_numpy(), label_strategy='threshold', threshold=0.2)
test_features, test_labels = make_sequences(test_features, labels_test.to_numpy(), label_strategy='threshold', threshold=0.2)

In [ ]:
class TrafficEnv:
    """Environment for sequential traffic inspection.
    Presents traffic feature windows one at a time and provides a reward based on correct or incorrect prediction."""
    def __init__(self, features: np.ndarray, labels: np.ndarray):
        """
        Initialize the environment with feature windows and corresponding labels.
        :param features: NumPy array of shape (N, state_dim) for states (traffic windows).
        :param labels: NumPy array of shape (N,) for binary labels (0 = benign, 1 = malicious).
        """
        self.features = features
        self.labels = labels
        self.n_samples = features.shape[0]
        self.index = 0  # pointer to current position in the sequence

    def reset(self):
        """Reset to the beginning of the sequence and return the first state."""
        self.index = 0
        return self.features[self.index]

    def step(self, action: int):
        """
        Take an action (0 or 1 prediction for the current window).
        Returns (next_state, reward, done) after applying the action.
        """
        true_label = self.labels[self.index]
        # Reward +1 for correct prediction, -1 for incorrect
        reward = 1.0 if action == true_label else -1.0

        # Move to the next state
        self.index += 1
        if self.index >= self.n_samples:
            # If we've reached the end of the sequence, episode is done
            done = True
            # For a terminal state, we can return a zero vector as next state (won't be used when done=True)
            next_state = torch.tensor(np.zeros_like(self.features[0].cpu()), device=device)
        else:
            done = False
            next_state = self.features[self.index]
        return next_state, reward, done


In [ ]:
class DRQN(nn.Module):
    def __init__(self, input_dim, hidden_dim, action_dim):
        super(DRQN, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, action_dim)
        self.hidden_dim = hidden_dim

    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        q_values = self.fc(out[:, -1, :])
        return q_values, hidden

    def init_hidden(self, batch_size, device):
        h0 = torch.zeros(1, batch_size, self.hidden_dim, device=device)
        c0 = torch.zeros(1, batch_size, self.hidden_dim, device=device)
        return (h0, c0)

In [ ]:
class ReplayMemory:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
def evaluate(env, net):
    net.eval()
    state, done = env.reset(), False
    hidden      = net.init_hidden(1, device)
    y_true, y_pred = [], []
    with torch.no_grad():
        while not done:
            st = state.unsqueeze(0)           # (1, seq_len, feat_dim)
            qvals, hidden = net(st, hidden)
            action       = int(qvals.argmax(dim=1).item())
            y_true.append(int(env.labels[env.index]))
            y_pred.append(action)
            state, _, done = env.step(action)
    net.train()
    return accuracy_score(y_true, y_pred)

In [ ]:
state_dim = train_features.shape[1]
hidden_dim = 128
action_dim = 2
learning_rate = 1e-3
gamma = 0.99
batch_size = 256
memory_capacity = 10000
epsilon_start = 1.0
epsilon_min = 0.1
epsilon_decay = 0.98
num_episodes = 100

seq_len = 20
input_dim = 53

train_features_gpu = torch.tensor(train_features, dtype=torch.float32, device=device)
train_labels_gpu   = torch.tensor(train_labels,   dtype=torch.int64,   device=device)

test_features_gpu = torch.tensor(test_features, dtype=torch.float32, device=device)
test_labels_gpu   = torch.tensor(test_labels,   dtype=torch.int64,   device=device)

env_train_eval = TrafficEnv(train_features_gpu, train_labels_gpu)
env_val        = TrafficEnv(test_features_gpu,  test_labels_gpu)

env = TrafficEnv(train_features_gpu, train_labels_gpu)
policy_net = DRQN(input_dim, hidden_dim, action_dim).to(device)
target_net = DRQN(input_dim, hidden_dim, action_dim).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
memory = ReplayMemory(memory_capacity)
epsilon = epsilon_start
eps = epsilon_start

outer = trange(num_episodes, desc="Training", unit="ep")
for ep in outer:
    state, done = env.reset(), False         # state: (seq_len, feat_dim) on GPU
    hidden      = policy_net.init_hidden(1, device)
    total_reward= 0.0

    inner = tqdm(total=env.n_samples, desc=f"Ep {ep+1}", leave=False, unit="step")
    while not done:
        # 1) build input for LSTM: (batch=1, seq_len, feat_dim)
        st = state.unsqueeze(0)

        # 2) choose action
        if random.random() < eps:
            a = random.randrange(action_dim)
        else:
            with torch.no_grad():
                qvals, hidden = policy_net(st, hidden)
                a             = int(qvals.argmax(dim=1).item())

        # 3) step env
        nxt, r, done = env.step(a)
        total_reward += r
        memory.push(state, a, r, nxt, done)
        state         = nxt

        # 4) training once we have enough samples
        if len(memory) >= batch_size:
            batch = memory.sample(batch_size)
            states, acts, rews, nexts, dones = zip(*batch)

            # stack into (B, seq_len, feat_dim)
            sb = torch.stack(states)                # already on GPU
            ns = torch.stack(nexts)

            ab = torch.tensor(acts, dtype=torch.int64, device=device).unsqueeze(1)
            rb = torch.tensor(rews, dtype=torch.float32, device=device).unsqueeze(1)
            db = torch.tensor(dones, dtype=torch.float32, device=device).unsqueeze(1)

            # current Q
            h0, c0    = policy_net.init_hidden(batch_size, device)
            qc, _     = policy_net(sb, (h0, c0))
            qc         = qc.gather(1, ab)

            # target Q (Double DQN)
            with torch.no_grad():
                nh0, nc0   = policy_net.init_hidden(batch_size, device)
                qn, _      = policy_net(ns, (nh0, nc0))
                best_a     = qn.argmax(dim=1, keepdim=True)

                th0, tc0   = target_net.init_hidden(batch_size, device)
                qt, _      = target_net(ns, (th0, tc0))
                qt_target  = rb + gamma * qt.gather(1, best_a) * (1 - db)

            loss = criterion(qc, qt_target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        inner.update(1)

    inner.close()
    # ----- end of episode: decay, sync -----
    eps = max(epsilon_min, eps * epsilon_decay)
    target_net.load_state_dict(policy_net.state_dict())

    # ----- compute train accuracy every episode -----
    train_acc = evaluate(env_train_eval, policy_net)

    # ----- compute val accuracy every 10 episodes -----
    val_acc = evaluate(env_val, policy_net)

    # ----- update outer tqdm postfix -----
    postfix = {"reward": f"{total_reward:.1f}", "train_acc": f"{train_acc:.4f}"}
    if val_acc is not None:
        postfix["val_acc"] = f"{val_acc:.4f}"
    outer.set_postfix(postfix)
outer.close()

# --- evaluation (same reshape logic) ---
env_test = TrafficEnv(test_features, test_labels)
state, done = env_test.reset(), False
hidden = policy_net.init_hidden(1, device)
y_true, y_pred = [], []

policy_net.eval()
with torch.no_grad():
    while not done:
        st = torch.tensor(state, dtype=torch.float32, device=device)
        st = st.view(1, seq_len, input_dim)
        qvals, hidden = policy_net(st, hidden)
        action = int(qvals.argmax(dim=1).item())

        y_true.append(int(env_test.labels[env_test.idx]))
        y_pred.append(action)
        state, _, done = env_test.step(action)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

print("precision", precision)
print("recall", recall)
print("f1-score", f1)
print("accuracy", accuracy)

Training:   0%|          | 0/100 [00:00<?, ?ep/s]

Ep 1:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 2:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 3:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 4:   0%|          | 0/53651 [00:00<?, ?step/s]

...

Ep 42:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 43:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 44:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 45:   0%|          | 0/53651 [00:00<?, ?step/s]

KeyboardInterrupt: 

In [ ]:
policy_net.train()

DRQN(
  (lstm): LSTM(53, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [ ]:
best_val = 0.9282258971762853

In [ ]:
outer = trange(66, desc="Training", unit="ep")
for ep in outer:
    state, done = env.reset(), False         # state: (seq_len, feat_dim) on GPU
    hidden      = policy_net.init_hidden(1, device)
    total_reward= 0.0

    inner = tqdm(total=env.n_samples, desc=f"Ep {ep+1}", leave=False, unit="step")
    while not done:
        # 1) build input for LSTM: (batch=1, seq_len, feat_dim)
        st = state.unsqueeze(0)

        # 2) choose action
        if random.random() < eps:
            a = random.randrange(action_dim)
        else:
            with torch.no_grad():
                qvals, hidden = policy_net(st, hidden)
                a             = int(qvals.argmax(dim=1).item())

        # 3) step env
        nxt, r, done = env.step(a)
        total_reward += r
        memory.push(state, a, r, nxt, done)
        state         = nxt

        # 4) training once we have enough samples
        if len(memory) >= batch_size:
            batch = memory.sample(batch_size)
            states, acts, rews, nexts, dones = zip(*batch)

            # stack into (B, seq_len, feat_dim)
            sb = torch.stack(states)                # already on GPU
            ns = torch.stack(nexts)

            ab = torch.tensor(acts, dtype=torch.int64, device=device).unsqueeze(1)
            rb = torch.tensor(rews, dtype=torch.float32, device=device).unsqueeze(1)
            db = torch.tensor(dones, dtype=torch.float32, device=device).unsqueeze(1)

            # current Q
            h0, c0    = policy_net.init_hidden(batch_size, device)
            qc, _     = policy_net(sb, (h0, c0))
            qc         = qc.gather(1, ab)

            # target Q (Double DQN)
            with torch.no_grad():
                nh0, nc0   = policy_net.init_hidden(batch_size, device)
                qn, _      = policy_net(ns, (nh0, nc0))
                best_a     = qn.argmax(dim=1, keepdim=True)

                th0, tc0   = target_net.init_hidden(batch_size, device)
                qt, _      = target_net(ns, (th0, tc0))
                qt_target  = rb + gamma * qt.gather(1, best_a) * (1 - db)

            loss = criterion(qc, qt_target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        inner.update(1)

    inner.close()
    # ----- end of episode: decay, sync -----
    eps = max(epsilon_min, eps * epsilon_decay)
    target_net.load_state_dict(policy_net.state_dict())

    # ----- compute train accuracy every episode -----
    train_acc = evaluate(env_train_eval, policy_net)

    # ----- compute val accuracy every 10 episodes -----
    val_acc = evaluate(env_val, policy_net)

    if val_acc > best_val:
        best_val = val_acc
        patience_ctr = 0
        torch.save(policy_net.state_dict(), "best_drqn.pt")

    # ----- update outer tqdm postfix -----
    postfix = {"reward": f"{total_reward:.1f}", "train_acc": f"{train_acc:.4f}"}
    if val_acc is not None:
        postfix["val_acc"] = f"{val_acc:.4f}"
    outer.set_postfix(postfix)
outer.close()

Training:   0%|          | 0/66 [00:00<?, ?ep/s]

Ep 1:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 2:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 3:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 4:   0%|          | 0/53651 [00:00<?, ?step/s]

...

Ep 45:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 46:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 47:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 48:   0%|          | 0/53651 [00:00<?, ?step/s]

Ep 49:   0%|          | 0/53651 [00:00<?, ?step/s]

In [ ]:
torch.save(policy_net.state_dict(), "best_drqn.pt")

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)

In [ ]:
class_names = ["Benign", "Attack"]

In [ ]:
env_test = TrafficEnv(test_features_gpu, test_labels_gpu)
state, done = env_test.reset(), False
hidden = policy_net.init_hidden(1, device)
y_true, y_pred = [], []

policy_net.eval()
with torch.no_grad():
    while not done:
        st = torch.tensor(state, dtype=torch.float32, device=device)
        st = st.view(1, seq_len, input_dim)
        qvals, hidden = policy_net(st, hidden)
        action = int(qvals.argmax(dim=1).item())

        y_true.append(int(env_test.labels[env_test.index]))
        y_pred.append(action)
        state, _, done = env_test.step(action)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

print("\nClassification Report:")
print(classification_report(
    y_true, y_pred, target_names=class_names, digits=4
))

# 4) Confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))

print("precision", precision)
print("recall", recall)
print("f1-score", f1)
print("accuracy", accuracy)
print(cm)

<ipython-input-80-1389643866>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  st = torch.tensor(state, dtype=torch.float32, device=device)



Classification Report:
              precision    recall  f1-score   support

      Benign     0.9251    0.9891    0.9560     14016
      Attack     0.9452    0.7015    0.8053      3762

    accuracy                         0.9282     17778
   macro avg     0.9351    0.8453    0.8807     17778
weighted avg     0.9293    0.9282    0.9241     17778

precision 0.9452005730659025
recall 0.7014885699096225
f1-score 0.8053097345132744
accuracy 0.9282258971762853
[[13863   153]
 [ 1123  2639]]
